In [1]:
import numpy as np
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def identity_block(X, ks, filters):
  f1, f2, f3 = filters
  #We are performing identity mapping after the 3 component blocks (instead of 2 - ResNet34) which include bottleneck 1x1 layers i.e F1 and F3 having number of filters as variable 'filters'.

  X_identity = X
  #We will add this later

  #CONV-BN-RELU x 2 - CONV-BN-IdentityMapping-RELU

  X = Conv2D(filters = f1, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
  #Glorot_uniform = Xavier Initialisation
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = f2, kernel_size = (ks, ks), strides = (1, 1), padding = 'same', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = f3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3)(X)
  X = Add()([X_identity, X])
  X = Activation('relu')(X)

  return X

In [3]:
def convolutional_block(X, ks, filters, stride = 2):
  f1, f2, f3 = filters

  X_identity = X

  #Structure same as identity block, but we do Conv and BN on identity mapping too

  X = Conv2D(f1, (1,1), strides = (stride, stride), kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = f2, kernel_size = (ks, ks), strides = (1, 1), padding = 'same', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = f3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)

  X_identity = Conv2D(filters = f3, kernel_size = (1, 1), strides = (stride, stride), padding = 'valid', kernel_initializer=glorot_uniform(seed=0))(X_identity)
  X_identity = BatchNormalization(axis = 3)(X_identity)
  
  X = Add()([X, X_identity])
  X = Activation('relu')(X)

  return X

The Structure of the ResNet50 is as follows

ZeroPad - (Conv - BN - ReLU - MaxPool) - (CB-IBx2) - (CB-IBx3) - (CB-IBx5) - (CB-IBx2) - Flatten - FC - Dropout - FC - Output

It usually is Avgpool - flatten - FC - output in the end but we're using Flatten FC Dropout FC Output because it gives better results

In [4]:
def ResNet50(input_shape = (64, 64, 3), classes = 10):
  X_input = Input(input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3, 3), strides = (2, 2))(X)

  X = convolutional_block(X, ks = 3, filters = [64, 64, 256], stride = 1)
  X = identity_block(X, 3, [64, 64, 256])
  X = identity_block(X, 3, [64, 64, 256])

  X = convolutional_block(X, ks = 3, filters = [128, 128, 512], stride = 2)
  X = identity_block(X, 3, [128, 128, 512])
  X = identity_block(X, 3, [128, 128, 512])

  X = convolutional_block(X, ks = 3, filters = [256, 256, 1024], stride = 2)
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])

  X = convolutional_block(X, ks = 3, filters = [512, 512, 2048], stride = 2)
  X = identity_block(X, 3, [512, 512, 2048])
  X = identity_block(X, 3, [512, 512, 2048])

  X = Flatten()(X)
  X = Dense(1024, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
  X = Dropout(0.5)(X)
  X = Dense(10, activation = 'softmax', kernel_initializer=glorot_uniform(seed=0))(X)

  model = Model(inputs = X_input, outputs = X, name = 'ResNet50')

  return model

In [5]:
model = ResNet50(input_shape = (64, 64, 3), classes = 10)

In [6]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(learning_rate = 1e-4), metrics = ['accuracy'])

In [7]:
#Image Augmentation

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
training_set = train_datagen.flow_from_directory('C:\\Users\\Musab Ahmed Pathan\\Documents\\EUROSAT\\Training Set', target_size = (64,64), batch_size=32, class_mode='categorical')
val_set = test_datagen.flow_from_directory('C:\\Users\\Musab Ahmed Pathan\\Documents\\EUROSAT\\Validation Set', target_size = (64,64), batch_size=32, class_mode='categorical')

Found 20600 images belonging to 10 classes.
Found 6400 images belonging to 10 classes.


In [9]:
#We save our progress periodically
filepath="C:\\Users\\Musab Ahmed Pathan\\Documents\\EUROSAT\\best_model4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Epoch 65 - val_accuracy improved from 0.90781 to 0.91297, saving model to C:\Users\Musab Ahmed Pathan\Documents\EUROSAT\best_model3.hdf5

In [10]:
model = load_model('C:\\Users\\Musab Ahmed Pathan\\Documents\\EUROSAT\\best_model4.hdf5')

In [11]:
EuroSAT_ResNet50 = model.fit(training_set, epochs=40, callbacks=callbacks_list, validation_data=val_set)

Epoch 1/40
644/644 [==============================] - 963s 1s/step - loss: 0.4893 - accuracy: 0.9380 - val_loss: 2.3823 - val_accuracy: 0.7245

Epoch 00001: val_accuracy improved from -inf to 0.72453, saving model to C:\Users\Musab Ahmed Pathan\Documents\EUROSAT\best_model4.hdf5
Epoch 2/40
644/644 [==============================] - 948s 1s/step - loss: 0.2903 - accuracy: 0.9426 - val_loss: 1.0873 - val_accuracy: 0.8444

Epoch 00002: val_accuracy improved from 0.72453 to 0.84438, saving model to C:\Users\Musab Ahmed Pathan\Documents\EUROSAT\best_model4.hdf5
Epoch 3/40
644/644 [==============================] - 941s 1s/step - loss: 0.2686 - accuracy: 0.9444 - val_loss: 0.4555 - val_accuracy: 0.9078

Epoch 00003: val_accuracy improved from 0.84438 to 0.90781, saving model to C:\Users\Musab Ahmed Pathan\Documents\EUROSAT\best_model4.hdf5
Epoch 4/40
644/644 [==============================] - 947s 1s/step - loss: 0.3622 - accuracy: 0.9492 - val_loss: 4.6768 - val_accuracy: 0.6120

Epoch 0000

In [11]:
preds = model.evaluate(val_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

200/200 [==============================] - 41s 188ms/step - loss: 0.3941 - accuracy: 0.9130
Loss = 0.3940958082675934
Test Accuracy = 0.9129687547683716
